# Binary star evolution and binary black holes

All the relevant information for the project are to be found in the pdf document present in the repo.
Note that you are assigned to project 1 (as the title said).

## Datasets 

Datasets are stored on Google Drive (link and description in the pdf document)

### Contacts

* Michela Mapelli <michela.mapelli@unipd.it>


In [1]:
import pandas as pd
import numpy as np
import string
import glob
import os

In [8]:
alpha_values = [0.5, 1, 3, 5]
met_values = [0.02, 0.002, 0.0002, 0.004, 0.0004, 0.006, 0.008, 0.012, 0.0012, 0.016, 0.0016]

# path = '/data/TheFormationOfBinaryBlackHoles_data/stable_MT_vs_CE' 

li = []
for alpha in alpha_values:
    for met in met_values:
            
        for name in glob.glob(f'/data/TheFormationOfBinaryBlackHoles_data/stable_MT_vs_CE/A{alpha}/MTCE_BBHs_{met}*'):
            df = pd.read_csv(name, skiprows=2, header=0, sep=' ')
            df = df.loc[:,['col.0:ID','col.1:m1ZAMS/Msun','col.3:m1rem/Msun','col.4:m2rem/Msun',  'col.6:delay_time/Myr', 'col.7:sma/Rsun', 'col.8:ecc', 'col.21:CE']]
            met_array = np.ones((df.shape[0], 1))*met
            alpha_array = np.ones((df.shape[0],1))*alpha
            df['metallicity'] = met_array
            df['alpha'] = alpha_array
            
        li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)


In [17]:
frame

,col.0:ID,col.1:m1ZAMS/Msun,col.3:m1rem/Msun,col.4:m2rem/Msun,col.6:delay_time/Myr,col.7:sma/Rsun,col.8:ecc,col.21:CE,metallicity,alpha
0,0_6942,50.9151,4.8001,4.7478,2233.6292,7.9524,0.273150,True,0.0200,0.5
1,0_80347,57.0253,5.5849,5.5325,1076.5703,9.0222,0.508320,True,0.0200,0.5
2,0_158611,91.0594,8.3787,8.2624,6810.4365,14.9810,0.072798,True,0.0200,0.5
3,0_211410,117.9396,19.3984,11.2422,7905.6343,11229.0000,0.999640,False,0.0200,0.5
4,0_213101,40.2965,3.1045,3.0769,8703.5479,13.7630,0.705290,True,0.0200,0.5
...,...,...,...,...,...,...,...,...,...,...
2443713,4_1998897,49.0692,16.2246,16.4283,1327.5031,16.3610,0.015313,False,0.0016,5.0
2443714,4_1999111,123.4732,38.2710,35.3085,6482.5015,44.8900,0.024311,False,0.0016,5.0
2443715,4_1999410,26.1787,4.3739,3.1328,1593.2186,6.9409,0.455930,True,0.0016,5.0
2443716,4_1999521,28.9126,5.8776,14.7098,12496.7129,19.4270,0.024287,True,0.0016,5.0


In [4]:
#MTCE_002_05_true = MTCE_002_05[(MTCE_002_05['col.21:CE'] == True)] 

In [5]:
#MTCE_002_05_true

In [6]:
#MTCE_002_05_false = MTCE_002_05[(MTCE_002_05['col.21:CE'] == False)] 
#MTCE_002_05_false